In [2]:
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/utils.py

!pip install -r requirements.txt
import gdown
url = 'https://drive.google.com/file/d/1Ns80_G8fAA0xs4e-7yvCVToYD-D5bNmb/view?usp=sharing'
output = 'params.py'
gdown.download(url=url, output=output, quiet=False, fuzzy=True)

--2024-12-07 01:02:30--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      82  --.-KB/s    in 0s      

2024-12-07 01:02:31 (1.00 MB/s) - ‘requirements.txt’ saved [82/82]

--2024-12-07 01:02:31--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6721 (6.6K) [text/plain]
Saving to

Downloading...
From (original): https://drive.google.com/uc?id=1Ns80_G8fAA0xs4e-7yvCVToYD-D5bNmb
From (redirected): https://drive.google.com/uc?id=1Ns80_G8fAA0xs4e-7yvCVToYD-D5bNmb&confirm=t&uuid=8a5a7a81-d3d3-4026-83da-9a373660e75c
To: /content/params.py
100%|██████████| 278/278 [00:00<00:00, 308kB/s]


'params.py'

In [3]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

import params
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

In [51]:
# Configurações de treinamento
train_config = SimpleNamespace(
    framework="sklearn",  # Ajustado para Regressão Logística (sklearn)
    lr=2e-3,              # Taxa de aprendizado para o modelo
    seed=42,              # Seed para reprodutibilidade
)


In [52]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=train_config)

wandb: Currently logged in as: abiapegado014 (abiapegado014-ufrn). Use `wandb login --relogin` to force relogin


In [53]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

wandb:   3 of 3 files downloaded.  


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


# Copiar o dataset para modificações
df_copy = df.copy()

# Converter a coluna 'classe' para valores numéricos
label_encoder = LabelEncoder()
df_copy['classe'] = label_encoder.fit_transform(df['classe'])

df_copy

,0,1,2,3,4,5,6,7,8,Stage,classe
0,sample_603,7,150,78,29,126,35.2,0.692,54,train,1
1,sample_555,7,124,70,33,215,25.5,0.161,37,train,0
2,sample_691,13,158,114,0,0,42.3,0.257,44,train,1
3,sample_725,4,112,78,40,0,39.4,0.236,38,train,0
4,sample_417,4,144,82,32,0,38.5,0.554,37,train,1
...,...,...,...,...,...,...,...,...,...,...,...
395,sample_104,2,85,65,0,0,39.6,0.930,27,valid,0
396,sample_446,1,100,72,12,70,25.3,0.658,28,valid,0
397,sample_141,5,106,82,30,0,39.5,0.286,38,valid,0
398,sample_260,3,191,68,15,130,30.9,0.299,34,valid,0


In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# Filtrar dados de treino, validação e teste usando as colunas 'Stage' e 'is_valid'
df_train = df_copy[df_copy['Stage'] == 'train']
df_valid = df_copy[df_copy['Stage'] == 'valid']
df_test = df_copy[df_copy['Stage'] == 'test']

# Separar as variáveis independentes (X) e dependentes (y)
X_train = df_train.drop(columns=['classe', '0', 'Stage'])
y_train = df_train['classe']

X_val = df_valid.drop(columns=['classe', '0', 'Stage'])
y_val = df_valid['classe']

X_test = df_test.drop(columns=['classe', '0', 'Stage'])
y_test = df_test['classe']


In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Normalizar os dados numéricos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Criar o modelo de Regressão Logística com balanceamento de classe
model = LogisticRegression(random_state=train_config.seed, max_iter=1000, class_weight='balanced')

# Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

# Fazer previsões no conjunto de validação e teste
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Avaliar o desempenho no conjunto de validação
val_accuracy = accuracy_score(y_val, y_val_pred)
val_report = classification_report(y_val, y_val_pred)

# Avaliar o desempenho no conjunto de teste
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred)

# Imprimir os resultados
print("==== Resultados - Validação ====")
print(f"Acurácia: {val_accuracy:.4f}")
print(val_report)

print("\n==== Resultados - Teste ====")
print(f"Acurácia: {test_accuracy:.4f}")
print(test_report)


==== Resultados - Validação ====
Acurácia: 0.8167
              precision    recall  f1-score   support

           0       0.91      0.70      0.79        30
           1       0.76      0.93      0.84        30

    accuracy                           0.82        60
   macro avg       0.83      0.82      0.81        60
weighted avg       0.83      0.82      0.81        60


==== Resultados - Teste ====
Acurácia: 0.5167
              precision    recall  f1-score   support

           0       1.00      0.03      0.06        30
           1       0.51      1.00      0.67        30

    accuracy                           0.52        60
   macro avg       0.75      0.52      0.37        60
weighted avg       0.75      0.52      0.37        60



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [71]:

predictions_df = pd.DataFrame({
    'id': df_test['0'],
    'Ground Truth': y_test,
    'Predictions': y_test_pred
})

# Criar uma tabela W&B
table = wandb.Table(dataframe=predictions_df)

# Logar a tabela no W&B
wandb.log({"prediction_table": table})

# Finalizar o log do W&B
wandb.finish()
